In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels
import numpy as np
import seaborn as sns
from scipy.stats import norm
from statsmodels.formula.api import logit
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

# Looking at the data

In [2]:
data = pd.read_csv("occupancy.csv")

In [3]:
data.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,02/02/2015 14:19,23.7000,26.272,585.200000,749.200000,0.004764,1
1,02/02/2015 14:19,23.7180,26.290,578.400000,760.400000,0.004773,1
2,02/02/2015 14:21,23.7300,26.230,572.666667,769.666667,0.004765,1
3,02/02/2015 14:22,23.7225,26.125,493.750000,774.750000,0.004744,1
4,02/02/2015 14:23,23.7540,26.200,488.600000,779.000000,0.004767,1


In [4]:
# Splitting the date column into date and time

data[["Date", "Time"]] = data.date.str.split(expand=True)
data.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Date,Time
0,02/02/2015 14:19,23.7000,26.272,585.200000,749.200000,0.004764,1,02/02/2015,14:19
1,02/02/2015 14:19,23.7180,26.290,578.400000,760.400000,0.004773,1,02/02/2015,14:19
2,02/02/2015 14:21,23.7300,26.230,572.666667,769.666667,0.004765,1,02/02/2015,14:21
3,02/02/2015 14:22,23.7225,26.125,493.750000,774.750000,0.004744,1,02/02/2015,14:22
4,02/02/2015 14:23,23.7540,26.200,488.600000,779.000000,0.004767,1,02/02/2015,14:23


In [5]:
# Drop the initial date column
data = data.drop(["date"], axis=1)
data.head()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Date,Time
0,23.7000,26.272,585.200000,749.200000,0.004764,1,02/02/2015,14:19
1,23.7180,26.290,578.400000,760.400000,0.004773,1,02/02/2015,14:19
2,23.7300,26.230,572.666667,769.666667,0.004765,1,02/02/2015,14:21
3,23.7225,26.125,493.750000,774.750000,0.004744,1,02/02/2015,14:22
4,23.7540,26.200,488.600000,779.000000,0.004767,1,02/02/2015,14:23


In [6]:
# Looking for the first occurence of a weekend begins on the 5914ith observation
data.iloc[5913:]

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Date,Time
5913,20.0000,18.7000,0.0,440.000000,0.002696,0,06/02/2015,23:58
5914,20.0000,18.7000,0.0,438.000000,0.002696,0,07/02/2015,00:00
5915,19.9725,18.7450,0.0,433.666667,0.002698,0,07/02/2015,00:01
5916,20.0000,18.7900,0.0,434.500000,0.002709,0,07/02/2015,00:02
5917,20.0000,18.7000,0.0,434.000000,0.002696,0,07/02/2015,00:03
...,...,...,...,...,...,...,...,...
10803,21.0500,36.0975,433.0,787.250000,0.005579,1,10/02/2015,09:29
10804,21.0500,35.9950,433.0,789.500000,0.005563,1,10/02/2015,09:29
10805,21.1000,36.0950,433.0,798.500000,0.005596,1,10/02/2015,09:30
10806,21.1000,36.2600,433.0,820.333333,0.005621,1,10/02/2015,09:32


In [7]:
# The weekend ends on the 8794th observation with the rest being weekdays
data.iloc[8792:]

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Date,Time
8792,19.50,27.1000,0.0,459.000000,0.003795,0,08/02/2015,23:57
8793,19.50,27.1000,0.0,458.000000,0.003795,0,08/02/2015,23:58
8794,19.50,27.1000,0.0,459.000000,0.003795,0,09/02/2015,00:00
8795,19.50,27.1000,0.0,459.000000,0.003795,0,09/02/2015,00:01
8796,19.50,27.1000,0.0,458.000000,0.003795,0,09/02/2015,00:02
...,...,...,...,...,...,...,...,...
10803,21.05,36.0975,433.0,787.250000,0.005579,1,10/02/2015,09:29
10804,21.05,35.9950,433.0,789.500000,0.005563,1,10/02/2015,09:29
10805,21.10,36.0950,433.0,798.500000,0.005596,1,10/02/2015,09:30
10806,21.10,36.2600,433.0,820.333333,0.005621,1,10/02/2015,09:32


In [8]:
# Setting a new binary column parameter known as day
data[["Day"]] = "weekday"
data.head()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Date,Time,Day
0,23.7000,26.272,585.200000,749.200000,0.004764,1,02/02/2015,14:19,weekday
1,23.7180,26.290,578.400000,760.400000,0.004773,1,02/02/2015,14:19,weekday
2,23.7300,26.230,572.666667,769.666667,0.004765,1,02/02/2015,14:21,weekday
3,23.7225,26.125,493.750000,774.750000,0.004744,1,02/02/2015,14:22,weekday
4,23.7540,26.200,488.600000,779.000000,0.004767,1,02/02/2015,14:23,weekday


In [9]:
# Setting the days that are weekends manually
data.loc[5914:8793, "Day"] = "weekend"
data.iloc[5913:8795]

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Date,Time,Day
5913,20.0000,18.700,0.0,440.000000,0.002696,0,06/02/2015,23:58,weekday
5914,20.0000,18.700,0.0,438.000000,0.002696,0,07/02/2015,00:00,weekend
5915,19.9725,18.745,0.0,433.666667,0.002698,0,07/02/2015,00:01,weekend
5916,20.0000,18.790,0.0,434.500000,0.002709,0,07/02/2015,00:02,weekend
5917,20.0000,18.700,0.0,434.000000,0.002696,0,07/02/2015,00:03,weekend
...,...,...,...,...,...,...,...,...,...
8790,19.5000,27.000,0.0,456.000000,0.003781,0,08/02/2015,23:55,weekend
8791,19.5000,27.100,0.0,456.000000,0.003795,0,08/02/2015,23:57,weekend
8792,19.5000,27.100,0.0,459.000000,0.003795,0,08/02/2015,23:57,weekend
8793,19.5000,27.100,0.0,458.000000,0.003795,0,08/02/2015,23:58,weekend


Here we are now left with a relatively cleaner dataframe in terms of date along with an added Day parameter from EDA.

In [10]:
data.to_csv("cleaned_occupancy.csv")